# Introduction

We recently announced our new open-source **Agents SDK**, designed to help you build agentic AI applications using a lightweight, easy-to-use package with minimal abstractions.

This cookbook demonstrates how you can leverage the Agents SDK in combination with Stripe's API to handle dispute management, a common operational challenge many businesses face. Specifically, we focus on two real-world scenarios:

1. **Company Mistake:**  
   A scenario where the company clearly made an error, such as failing to fulfill an order, where accepting the dispute the appropriate action.

2. **Customer Dispute (Final Sale):**  
   A scenario where a customer knowingly disputes a transaction despite receiving the correct item and understanding that the purchase was final sale, requiring further investigation to gather supporting evidence.

To address these scenarios, we'll introduce three distinct agents:

- **Triage Agent:**  
  Determines whether to accept or escalate a dispute based on the fulfillment status of the order.

- **Acceptance Agent:**  
  Handles clear-cut cases by automatically accepting disputes, providing concise reasoning.

- **Investigator Agent:**  
  Conducts deeper investigations into disputes by examining customer emails, verifying IP addresses, shipping addresses, and other granular details to gather the necessary evidence.

Throughout this cookbook, we’ll guide you step-by-step, illustrating how custom agentic workflows can automate dispute management and support your business operations.


## Prerequisites

Before running this cookbook, you must set up the following accounts and complete a few setup actions. These prerequisites are essential to interact with the APIs used in this project.

#### 1. OpenAI Account

- **Purpose:**  
  You need an OpenAI account to access language models and use the Agents SDK featured in this cookbook.

- **Action:**  
  [Sign up for an OpenAI account](https://openai.com) if you don’t already have one. Once you have an account, create an API key by visiting the [OpenAI API Keys page](https://platform.openai.com/api-keys).

#### 2. Stripe Account

- **Purpose:**  
  A Stripe account is required to simulate payment processing, manage disputes, and interact with the Stripe API as part of our demo workflow.

- **Action:**  
  Create a free Stripe account by visiting the [Stripe Signup Page](https://dashboard.stripe.com/register).

- **Locate Your API Keys:**  
  Log in to your Stripe dashboard and navigate to **Developers > API keys**.

- **Use Test Mode:**  
  Use your **Test Secret Key** for all development and testing.


#### 3. Create a .env file with your OpenAI API and Stripe API Keys

```
OPENAI_API_KEY=
STRIPE_SECRET_KEY=
```

### Environment Setup
First we will install the necessary dependencies, then import the libraries and write some utility functions that we will use later on.

In [1]:
%pip install quart --quiet
%pip install python-dotenv --quiet
%pip install openai-agents --quiet
%pip install stripe --quiet
%pip install nest_asyncio --quiet
%pip install typing_extensions --quiet


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note

In [2]:
import os
import logging
import asyncio
import json
from dotenv import load_dotenv
from quart import Quart, request
from agents import Agent, Runner, function_tool  # Only import what you need
import stripe
from typing_extensions import TypedDict, Any
import nest_asyncio
import ipywidgets as widgets
from IPython.display import display
nest_asyncio.apply()
# Load environment variables from .env file
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set Stripe API key from environment variables
stripe.api_key = os.getenv("STRIPE_SECRET_KEY")

#Create the Quart Application
app = Quart(__name__)

#### Define Function Tools
This section defines several helper function tools that support the dispute processing workflow. 
<br>
 
- `get_zip_code` maps a given IP address to its corresponding zip code using a predefined dictionary.
- `get_order` and `get_emails` simulate external data lookups by returning order details and email records based on provided identifiers.
- `retrieve_payment_intent`  interacts with the Stripe API to fetch payment intent details.
- `close_dispute`  automatically closes a Stripe dispute using the provided dispute ID, ensuring that disputes are properly resolved and logged.


In [3]:
@function_tool
def get_zip_code(ip_address: str) -> str:
    """
    Return the zip code corresponding to the given IP address.
    If the IP is not found, return 'Zip code not found'.
    """
    ip_to_zip = {
        "192.168.1.1": "10001",
        "192.168.1.2": "10002",
        "192.168.1.3": "10003",
        "10.0.0.1": "20001",
        "10.0.0.2": "20002"
    }
    return ip_to_zip.get(ip_address, "Zip code not found")


@function_tool
def get_order(order_id: int) -> str:
    """
    Retrieve an order by ID from a predefined list of orders.
    Returns the corresponding order object or 'No order found'.
    """
    orders = [
        {
            "order_id": 1234,
            "fulfillment_details": "not_shipped"
        },
        {
            "order_id": 9101,
            "fulfillment_details": "shipped",
            "tracking_info": {
                "carrier": "FedEx",
                "tracking_number": "123456789012"
            },
            "delivery_status": "out for delivery"
        },
        {
            "order_id": 1121,
            "fulfillment_details": "delivered",
            "customer_id": "cus_PZ1234567890",
            "order_date": "2023-01-01",
            "customer_email": "customer1@example.com",
            "tracking_info": {
                "carrier": "UPS",
                "tracking_number": "1Z999AA10123456784",
                "delivery_status": "delivered"
            },
            "shipping_address": {
                "zip": "10001"
            },
            "tos_acceptance": {
                "date": "2023-01-01",
                "ip": "192.168.1.1"
            }
        }
    ]
    for order in orders:
        if order["order_id"] == order_id:
            return order
    return "No order found"


@function_tool
def get_emails(email: str) -> list:
    """
    Return a list of email records for the given email address.
    """
    emails = [
        {
            "email": "customer1@example.com",
            "subject": "Order #1121",
            "body": "Hey, I know you don't accept refunds but the sneakers don't fit and I'd like a refund"
        },
        {
            "email": "customer2@example.com",
            "subject": "Inquiry about product availability",
            "body": "Hello, I wanted to check if the new model of the smartphone is available in stock."
        },
        {
            "email": "customer3@example.com",
            "subject": "Feedback on recent purchase",
            "body": "Hi, I recently purchased a laptop from your store and I am very satisfied with the product. Keep up the good work!"
        }
    ]
    return [email_data for email_data in emails if email_data["email"] == email]


@function_tool
async def retrieve_payment_intent(payment_intent_id: str) -> dict:
    """
    Retrieve a Stripe payment intent by ID.
    Returns the payment intent object on success or an empty dictionary on failure.
    """
    try:
        return stripe.PaymentIntent.retrieve(payment_intent_id)
    except stripe.error.StripeError as e:
        logger.error(f"Stripe error occurred while retrieving payment intent: {e}")
        return {}

@function_tool
async def close_dispute(dispute_id: str) -> dict:
    """
    Close a Stripe dispute by ID. 
    Returns the dispute object on success or an empty dictionary on failure.
    """
    try:
        return stripe.Dispute.close(dispute_id)
    except stripe.error.StripeError as e:
        logger.error(f"Stripe error occurred while closing dispute: {e}")
        return {}


### Define the Agents

- The **Dispute Intake Agent (investigator_agent)** is responsible for investigating disputes by gathering all relevant evidence, such as retrieving customer emails and verifying that the IP address used for TOS acceptance matches the shipping address zip code.
- The **Accept a Dispute Agent (accept_dispute_agent)** handles disputes that are determined to be valid by automatically closing them and providing a brief explanation for the decision.
- The **Triage Agent (triage_agent)** serves as the decision-maker by extracting the order ID from the payment intent's metadata, retrieving detailed order information, and then deciding whether to escalate the dispute to the investigator or to pass it to the accept dispute agent.
- Together, these agents form a modular workflow that automates and streamlines the dispute resolution process by delegating specific tasks to specialized agents.


In [4]:
investigator_agent = Agent(
    name="Dispute Intake Agent",
    instructions=(
        "In your role as a dispute investigator, thoroughly investigate the dispute, gather all relevant evidence, "
        "identify any customer emails and provide their body text, and confirm that the IP address used to "
        "accept the Terms of Service matches the shipping address zip code."
    ),
    tools=[get_emails, get_zip_code]
)

accept_dispute_agent = Agent(
    name="Accept a Dispute",
    instructions="Close the dispute using the given id. Provide a short explanation, for example: 'I accepted this dispute because the order hasn't been fulfilled yet with a reference to the order that was retrieved from the database.'",
    tools=[close_dispute]
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Find the order id in the payment intent's metadata, retrieve more details about the order, and if "
                 "the order was shipped, escalate to the investigator agent. Otherwise, accept the dispute.",
    tools=[retrieve_payment_intent, get_order],
    handoffs=[accept_dispute_agent, investigator_agent],
)

#### Retrieve the Dispute and Initiate the Agentic Workflow
This function retrieves the dispute details from Stripe using the provided `payment_intent_id` and initiates the dispute-handling workflow by passing the retrieved dispute information to the specified `triage_agent`.


In [5]:
async def process_dispute(payment_intent_id, triage_agent):
    """Retrieve and process dispute data for a given PaymentIntent."""
    disputes_list = stripe.Dispute.list(payment_intent=payment_intent_id)
    if not disputes_list.data:
        logger.warning("No dispute data found for PaymentIntent: %s", payment_intent_id)
        return None
    
    dispute_data = disputes_list.data[0]
    
    relevant_data = {
        "dispute_id": dispute_data.get("id"),
        "amount": dispute_data.get("amount"),
        "due_by": dispute_data.get("evidence_details", {}).get("due_by"),
        "payment_intent": dispute_data.get("payment_intent"),
        "reason": dispute_data.get("reason"),
        "status": dispute_data.get("status"),
        "card_brand": dispute_data.get("payment_method_details", {}).get("card", {}).get("brand")
    }
    
    logger.info("Charge dispute created event received")
    event_str = json.dumps(relevant_data)
    logger.info("Relevant dispute data: %s", event_str)
    
    # Pass the dispute data to the triage agent
    result = await Runner.run(triage_agent, input=event_str)
    logger.info("Triage agent result: %s", result.final_output)
    
    return relevant_data, result.final_output

#### Scenario 1: Company Mistake (Product Not Received)
This scenario represents a situation where the company has clearly made an error—for instance, failing to fulfill or ship an order. In such cases, it may be appropriate to accept the dispute rather than contest it.

In [6]:
payment = stripe.PaymentIntent.create(
  amount=2000,
  currency="usd",
  payment_method = "pm_card_createDisputeProductNotReceived",
  confirm=True,
  metadata={"order_id": "1234"},
  off_session=True,
  automatic_payment_methods={"enabled": True},
)
relevant_data, triage_result = await process_dispute(payment.id, triage_agent)

INFO:stripe:message='Request to Stripe api' method=post url=https://api.stripe.com/v1/payment_intents
INFO:stripe:message='Stripe API response' path=https://api.stripe.com/v1/payment_intents response_code=200
INFO:stripe:message='Request to Stripe api' method=get url=https://api.stripe.com/v1/disputes?payment_intent=pi_3R44fCIHWlrjZQHr0FT8FzYU
INFO:stripe:message='Stripe API response' path=https://api.stripe.com/v1/disputes?payment_intent=pi_3R44fCIHWlrjZQHr0FT8FzYU response_code=200
INFO:__main__:Charge dispute created event received
INFO:__main__:Relevant dispute data: {"dispute_id": "dp_1R44fDIHWlrjZQHr6Rjec4Ne", "amount": 2000, "due_by": 1743119999, "payment_intent": "pi_3R44fCIHWlrjZQHr0FT8FzYU", "reason": "product_not_received", "status": "needs_response", "card_brand": "visa"}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:stripe:message='Request to Stripe api' method=get url=https://api.stripe.com/v1/payment_intents/pi_3R44fCIHWlrjZQHr0

#### Scenario 2: Customer Dispute (Final Sale)
This scenario respresents a situation where a customer knowingly disputes a transaction despite receiving the correct product and understanding clearly that the purchase was marked as "final sale" (no refunds or returns). Such disputes usually require additional investigation, gathering, and submission of evidence to contest the dispute.

In [7]:
payment = stripe.PaymentIntent.create(
  amount=2000,
  currency="usd",
  payment_method = "pm_card_createDispute",
  confirm=True,
  metadata={"order_id": "1121"},
  off_session=True,
  automatic_payment_methods={"enabled": True},
)
relevant_data, triage_result = await process_dispute(payment.id, triage_agent)

INFO:stripe:message='Request to Stripe api' method=post url=https://api.stripe.com/v1/payment_intents
INFO:stripe:message='Stripe API response' path=https://api.stripe.com/v1/payment_intents response_code=200
INFO:stripe:message='Request to Stripe api' method=get url=https://api.stripe.com/v1/disputes?payment_intent=pi_3R44fNIHWlrjZQHr1cvycN9L
INFO:stripe:message='Stripe API response' path=https://api.stripe.com/v1/disputes?payment_intent=pi_3R44fNIHWlrjZQHr1cvycN9L response_code=200
INFO:__main__:Charge dispute created event received
INFO:__main__:Relevant dispute data: {"dispute_id": "dp_1R44fOIHWlrjZQHrlD1GIZoE", "amount": 2000, "due_by": 1743119999, "payment_intent": "pi_3R44fNIHWlrjZQHr1cvycN9L", "reason": "fraudulent", "status": "needs_response", "card_brand": "visa"}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:stripe:message='Request to Stripe api' method=get url=https://api.stripe.com/v1/payment_intents/pi_3R44fNIHWlrjZQHr1cvycN9L
IN